In [53]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [64]:
# read from melodyData.txt
with open('out/melodyData.txt', 'r') as f:
    parts = eval(f.read())
# a =[(0, 0.25), (0, 1.0), (0, 3.0), (70, ...]


# one-hot encoding

encodings = {}
encodingIndex = 1
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


160


In [91]:
sequence_length = 10

X = []
Y = []
# given data_encoded, generate training data by looping 
for i in range(len(data_encoded)):
    for j in range(len(data_encoded[i]) - sequence_length):
        X.append(data_encoded[i][j:j + 10])
        Y.append(data_encoded[i][j + 10])

# split this into training and testing sets 
# we will intentionally overfit, GET RID OF THIS LATER
# TODO: separate different parts into train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# TODO: make the model itself
# benchmark: dense is 80% accuracy
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(sequence_length,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(encodings) + 1, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(Y_train), epochs=200)

Epoch 1/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0069 - loss: 11.7624    
Epoch 2/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0308 - loss: 5.1160   
Epoch 3/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0800 - loss: 4.7487 
Epoch 4/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0848 - loss: 4.4384
Epoch 5/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0877 - loss: 4.2415   
Epoch 6/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1119 - loss: 4.0663
Epoch 7/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1232 - loss: 3.8401
Epoch 8/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1620 - loss: 3.7160
Epoch 9/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1703 - loss: 3.5713 
Epoch 10/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1947 - loss: 3.4024
Epoch 11/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2319 - loss: 3.2566
Epoch 12/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [86]:
# given a sequence, generate the next note (up to ten times)
seq = X_test[0]
total_seq = seq[:]
for i in range(6):
    pred = model.predict(np.array([seq]))
    pred = np.argmax(pred)
    seq = seq[1:] + [pred]

    total_seq.append(pred)

mapping = {0: "C", 1: "C#", 2: "D", 3: "D#", 4: "E", 5: "F", 6: "F#", 7: "G", 8: "G#", 9: "A", 10: "A#", 11: "B"}

for note in total_seq:
    pitch, dur = decodings[note]
    print (mapping[pitch % 12], dur)

# convert numbers to musical note letters
# 60 = C

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
D 1.5
D 0.5
D 1.0
E 0.5
F# 0.5
G 1.5
G 0.5
D 2.0
G 1.5
G 0.5
C 2.0
G 2.0
G 2.0
G 1.5
E 0.5
G 1.5


In [79]:
l = []
for i in range(10):
    l.append(i)
print (l)

a = [i*i for i in l]
print (a)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
